In [4]:

import pandas as pd
import numpy as np
import random
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix


In [5]:
# mount my google drive to this notebook so I can access my files

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Stroke_Prediction/train_2v.csv")


# https://www.kaggle.com/datasets/shashwatwork/dementia-prediction-dataset

In [10]:
display(df)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43395,56196,Female,10.0,0,0,No,children,Urban,58.64,20.4,never smoked,0
43396,5450,Female,56.0,0,0,Yes,Govt_job,Urban,213.61,55.4,formerly smoked,0
43397,28375,Female,82.0,1,0,Yes,Private,Urban,91.94,28.9,formerly smoked,0
43398,27973,Male,40.0,0,0,Yes,Private,Urban,99.16,33.2,never smoked,0


In [8]:



def to_list(df_dict, heads):
    # {key: [], ...}
    out = {}
    for _key in list(heads):
        out[_key] = [df_dict[_key][key] for key in df_dict[_key].keys()]
    return out

def encode_input(data_range, data):
    return data_range.index(data)

def preprocess_data(df_list: dict, need_encoded: list, normalize=True) -> np.ndarray:
    output = []
    data_length = len(df_list['id'])
    data_ranges = {k : list(set(df_list[k])) for k in need_encoded}
    for i in range(data_length):
        item = []
        for key in list(df_list.keys())[1:-1]:
            if key in need_encoded:
                item.append(encode_input(data_ranges[key], df_list[key][i]))
            else:
                item.append(df_list[key][i])
        output.append(item)
    inputs_arr = np.array(output)
    targets_arr = np.array(df_list['stroke'])
    if normalize:
        _range = np.max(inputs_arr, axis=0) - np.min(inputs_arr, axis=0)
        inputs_arr = (inputs_arr-np.min(inputs_arr, axis=0)) / _range
    return inputs_arr, targets_arr

def prepare_data(inputs, targets, seed=1001):
    positive_mask = targets == 1
    negative_mask = targets == 0
    n_minimum = min(np.sum(positive_mask), np.sum(negative_mask))
    positive_inputs = inputs[positive_mask][0:n_minimum, :]
    positive_targets = targets[positive_mask][0:n_minimum]
    negative_inputs = inputs[negative_mask][0:n_minimum, :]
    negative_targets = targets[negative_mask][0:n_minimum]
    inputs = np.concatenate([positive_inputs, negative_inputs]).tolist()
    targets = np.concatenate([positive_targets, negative_targets]).tolist()
    np.random.seed(seed)
    np.random.shuffle(inputs)
    np.random.seed(seed)
    np.random.shuffle(targets)
    return np.array(inputs), np.array(targets)

def metrics(y_pred, y_true):
    _confusion_matrix = confusion_matrix(y_pred, y_true)
    tp = _confusion_matrix[0,0]
    fn = _confusion_matrix[1,0]
    fp = _confusion_matrix[0,1]
    tn = _confusion_matrix[1,1]
    # metrics
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fscore = 2*tp/(2*tp + fp + fn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    miss_rate = fn/(tn+tp)
    fall_out_rate = fp/(fp+tn)
    # return 
    return [precision, recall, fscore, accuracy, miss_rate, fall_out_rate]
df_clear = df.dropna(axis=0)
df_dict = df_clear.to_dict()
heads = list(df_dict.keys())
df_list = to_list(df_dict, heads)
need_encoded = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
# MLP 
n_repeat = 5
_metrics = []


In [9]:

for i in range(n_repeat):
    # generate data
    seed = random.randint(1, 1000)
    inputs, targets = preprocess_data(df_list, need_encoded)
    inputs, targets = prepare_data(inputs, targets, seed)
    n_samples = inputs.shape[0]
    tr_inputs = inputs[0:int(n_samples*0.7), :]
    tr_targets = targets[0:int(n_samples*0.7)]
    te_inputs = inputs[int(n_samples*0.7):, :]
    te_targets = targets[int(n_samples*0.7):]
    # train clf
    clf = MLPClassifier(learning_rate_init=1e-2,
        solver='sgd',activation='relu',max_iter=500,
        alpha=5e-4, hidden_layer_sizes=(16, 32, 16),random_state=1)
    clf.fit(tr_inputs, tr_targets)
    # evaluate clf
    te_pred = clf.predict(te_inputs)
    run_metrics = metrics(te_pred, te_targets)
    print(i+1, "complete", run_metrics)
    _metrics.append(run_metrics)

_metrics = np.array(_metrics)
_metrics = np.mean(_metrics, axis=0)
# [precision, recall, fscore, accuracy, miss_rate, fall_out_rate]
print(_metrics)

1 complete [0.795774647887324, 0.743421052631579, 0.7687074829931972, 0.7933130699088146, 0.14942528735632185, 0.1638418079096045]
2 complete [0.8333333333333334, 0.6748466257668712, 0.7457627118644068, 0.7720364741641338, 0.20866141732283464, 0.13253012048192772]
3 complete [0.7070063694267515, 0.7115384615384616, 0.7092651757188498, 0.723404255319149, 0.18907563025210083, 0.2658959537572254]
4 complete [0.7453416149068323, 0.75, 0.7476635514018691, 0.7537993920972644, 0.16129032258064516, 0.24260355029585798]
5 complete [0.78125, 0.7440476190476191, 0.7621951219512195, 0.7629179331306991, 0.17131474103585656, 0.21739130434782608]
[0.77254119 0.72477075 0.74671881 0.76109422 0.17595348 0.20445255]
